In [1]:
import os, sys

import numpy as np
import pandas as pd
import tensorflow as tf
import random

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
  sys.path.append(module_path)

from utils.utils import Create_Similarity_Model

2022-07-06 14:01:22.961067: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-06 14:01:22.961087: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Now we will create our model

In [2]:
n_nonsimilar = 1
n_similar = 1

### We will look for similarity in the last n_similar parameters of the inputs.

model = Create_Similarity_Model(n_nonsimilar = n_nonsimilar, n_similar = n_similar)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['input_1[0][0]']                
                                                                                                  
 lambda_1 (Lambda)              (None, 1)            0           ['input_1[0][0]']                
                                                                                                  
 similarity_layer_1 (Dense)     (None, 1)            1           ['lambda[0][0]']                 
                                                                                              

2022-07-06 14:01:24.193471: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 14:01:24.194006: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-06 14:01:24.194075: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-07-06 14:01:24.194121: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-07-06 14:01:24.194177: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

## Define the loss, optimizer and metrics

In [3]:
loss = 'mean_squared_error'
optimizer = tf.keras.optimizers.Adam(
    learning_rate= 0.0001,  #0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
)

metrics = ['mean_squared_error']

model.compile(loss = loss, optimizer = optimizer, metrics = metrics)

## Load the MVP data for high enough Re

In [4]:
dfSB2M300k_M=pd.read_csv("../../Data/McKeon Data/turb2/Re2300000.txt",  sep="\t", header=None, names = ["datapoint", "y/R", "y+", "U+", "(U+-Ucl+)"])
dfSB3M_M=pd.read_csv("../../Data/McKeon Data/turb2/Re3000000.txt", sep="\t", header=None, names = ["datapoint", "y/R", "y+", "U+", "(U+-Ucl+)"])
dfSB4M_M=pd.read_csv("../../Data/McKeon Data/turb2/Re4000000.txt", sep="\t", header=None, names = ["datapoint", "y/R", "y+", "U+", "(U+-Ucl+)"])
dfSB6M_M=pd.read_csv("../../Data/McKeon Data/turb2/Re6000000.txt", sep="\t", header=None, names = ["datapoint", "y/R", "y+", "U+", "(U+-Ucl+)"])
dfSB7M_M=pd.read_csv("../../Data/McKeon Data/turb2/Re7000000.txt", sep="\t", header=None, names = ["datapoint", "y/R", "y+", "U+", "(U+-Ucl+)"])
dfSB10M_M=pd.read_csv("../../Data/McKeon Data/turb2/Re10000000.txt", sep="\t", header=None, names = ["datapoint", "y/R", "y+", "U+", "(U+-Ucl+)"])
#dfSB13M_M=pd.read_csv("../../Data/McKeon Data/turb2/Re13000000.txt", sep="\t", header=None, names = ["datapoint", "y/R", "y+", "U+", "(U+-Ucl+)"])

dfs = [dfSB2M300k_M, dfSB3M_M, dfSB4M_M, dfSB6M_M, dfSB7M_M, dfSB10M_M] #, dfSB13M_M]
Re_tau = []
Y_ = []
U_ = []
possible_re_tau = [4.229500e+004, 5.453000e+004, 7.647800e+004, 1.022e+005, 1.279200e+005, 1.657e+005] #, 2.169800e+005]
dic_x = {}
dic_y = {}

for i in range(len(possible_re_tau)):
    possible_y = dfs[i]["y+"].values
    possible_u = dfs[i]["U+"].values
    for j in range(38, len(possible_y)-1):
        Y_.append(possible_y[j])
        U_.append(possible_u[j])
        Re_tau.append(possible_re_tau[i])

c = list(zip(Y_, Re_tau, U_))
random.shuffle(c)
Y_, Re_tau, U_ = zip(*c)

Y_ = np.log(Y_)
Re_tau = np.log(Re_tau)

dic_x['y+'] = Y_
dic_x['re_tau'] = Re_tau
dic_y['u+'] = U_

nonsimilar_keys = ['y+']

xtrain = pd.DataFrame.from_dict(dic_x).values
ytrain = pd.DataFrame.from_dict(dic_y).values

print(xtrain)
print(ytrain)

[[10.14940969 11.53468696]
 [10.79620255 11.75916035]
 [10.20714139 12.0179342 ]
 [10.15832355 10.90650629]
 [10.54278554 10.65242415]
 [10.95049071 12.0179342 ]
 [10.32996371 10.65242415]
 [ 9.09367172 10.90650629]
 [10.43640729 10.65242415]
 [11.27008928 12.0179342 ]
 [10.25467245 11.53468696]
 [10.15824604 11.75916035]
 [ 9.05294945 10.65242415]
 [ 8.83955201 10.65242415]
 [10.90334708 10.90650629]
 [11.00957186 11.75916035]
 [10.39203689 11.2447584 ]
 [10.69055759 10.90650629]
 [10.47585032 11.75916035]
 [ 9.43556176 11.2447584 ]
 [10.84665388 12.0179342 ]
 [10.79695991 10.90650629]
 [ 9.41336285 10.90650629]
 [11.33004775 11.75916035]
 [11.00171643 11.53468696]
 [ 9.20014861 10.90650629]
 [10.22343108 10.65242415]
 [12.01545679 12.0179342 ]
 [11.22286724 11.75916035]
 [11.48337306 12.0179342 ]
 [11.13798471 11.2447584 ]
 [10.37108203 10.90650629]
 [11.03110833 11.2447584 ]
 [ 9.30710404 10.90650629]
 [ 9.519368   10.90650629]
 [11.11752454 11.75916035]
 [ 9.73246196 10.90650629]
 

In [5]:
model.fit(xtrain, ytrain, epochs = 1000, verbose = 2)

Epoch 1/1000
4/4 - 0s - loss: 6244620288.0000 - mean_squared_error: 6244620288.0000 - 384ms/epoch - 96ms/step
Epoch 2/1000
4/4 - 0s - loss: 816113792.0000 - mean_squared_error: 816113792.0000 - 4ms/epoch - 997us/step
Epoch 3/1000
4/4 - 0s - loss: 972.5896 - mean_squared_error: 972.5896 - 5ms/epoch - 1ms/step
Epoch 4/1000
4/4 - 0s - loss: 972.5896 - mean_squared_error: 972.5896 - 5ms/epoch - 1ms/step
Epoch 5/1000
4/4 - 0s - loss: 972.5896 - mean_squared_error: 972.5896 - 7ms/epoch - 2ms/step
Epoch 6/1000
4/4 - 0s - loss: 972.5895 - mean_squared_error: 972.5895 - 4ms/epoch - 1ms/step
Epoch 7/1000
4/4 - 0s - loss: 972.5895 - mean_squared_error: 972.5895 - 4ms/epoch - 1ms/step
Epoch 8/1000
4/4 - 0s - loss: 972.5897 - mean_squared_error: 972.5897 - 4ms/epoch - 1ms/step
Epoch 9/1000
4/4 - 0s - loss: 972.5896 - mean_squared_error: 972.5896 - 5ms/epoch - 1ms/step
Epoch 10/1000
4/4 - 0s - loss: 972.5896 - mean_squared_error: 972.5896 - 4ms/epoch - 1ms/step
Epoch 11/1000
4/4 - 0s - loss: 972.589

In [6]:
for i in range(n_nonsimilar):
    similarity_exponents = model.get_layer('similarity_layer_' + str(i+1)).weights[0][0].numpy()
    print('Exponents multiplicating inside the function for ' + nonsimilar_keys[i] + ': ' + str(similarity_exponents))

multiplication_exponents = model.get_layer('multiplication_layer').weights[0][0].numpy()
print('Exponents multiplicating outside the function: ' + str(multiplication_exponents))

Exponents multiplicating inside the function for y+: [0.63157386]
Exponents multiplicating outside the function: [-0.27754205]
